In [ ]:
from typing import TypedDict
from langgraph.graph import StateGraph, END
from langchain_openai import ChatOpenAI
from IPython.display import Image, display


MODEL = "gpt-4.1-mini"
answer_llm = ChatOpenAI(model=MODEL, temperature=0.3)
judge_llm = ChatOpenAI(model=MODEL, temperature=0.0)

class State(TypedDict):
    question: str
    answer: str
    score: float
    attempts: int

def answer(state: State):
    msg = answer_llm.invoke(f"Answer in 5 bullets max:\n{state['question']}")
    return {"answer": msg.content, "attempts": state.get("attempts", 0) + 1}

def score(state: State):
    msg = judge_llm.invoke(
        "Score this answer from 0 to 1 for correctness + completeness. Output ONLY a number.\n\n"
        f"Q: {state['question']}\n\nA:\n{state['answer']}"
    )
    try:
        s = float(msg.content.strip())
    except:
        s = 0.5
    return {"score": max(0.0, min(1.0, s))}

def improve(state: State):
    msg = answer_llm.invoke(
        "Improve the answer to be more correct/complete but still concise.\n\n"
        f"Question: {state['question']}\n\nCurrent answer:\n{state['answer']}"
    )
    return {"answer": msg.content}

def route(state: State):
    # stop if good enough or too many attempts
    #state["score"] >= 1.1 or 
    if  state["score"] >= 1.0 or  state["attempts"] >= 2:
        return END
    return "improve"

g = StateGraph(State)
g.add_node("answer", answer)
g.add_node("score", score)
g.add_node("improve", improve)

g.set_entry_point("answer")
g.add_edge("answer", "score")
g.add_conditional_edges("score", route, {END: END, "improve": "improve"})
g.add_edge("improve", "score")  # loop back to rescore

graph = g.compile()

out = graph.invoke({"question": "What is an Agentic AI and AI agents", "attempts": 0})
print("Attempts:", out["attempts"])
print("Score:", out["score"])
print(out["answer"])

png_bytes = graph.get_graph().draw_mermaid_png()
display(Image(png_bytes))

KeyboardInterrupt: 